## Creating sub-agent 1: Fundamental Analyst agent

In this folder we will create the first sub-agent for getting our `input statements`, `balance sheets` and `cash flow statements`. This code defines an agent for retrieving income statements for publicly traded companies through a financial API service.

This `fundamental analyst agent` will be associated with an action group which will have access to three functions. Each of the function is responsible for performing certain tasks related to getting input statements of a company's stock based on the `ticker` symbol, balance sheets and other cash flow details. 

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [1]:
# Uninstall the current version of boto3, botocore and awscli, to install the updated version of boto3
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.35.83
Uninstalling boto3-1.35.83:
  Successfully uninstalled boto3-1.35.83
Found existing installation: botocore 1.35.83
Uninstalling botocore-1.35.83:
  Successfully uninstalled botocore-1.35.83
Found existing installation: awscli 1.36.24
Uninstalling awscli-1.36.24:
  Successfully uninstalled awscli-1.36.24


In [2]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Importing helper functions

On following section, we're adding `agents.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `agent.py`.

Those files contain helper classes totally focused on make labs experience smoothly.

All interactions with Bedrock will be handled by these classes.

#### Following are methods that you're going to invoke on this lab:

- `create_agent`: Create a new agent and respective IAM roles

- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent

- `create_agent_alias`: Create an alias for this agent

- `invoke`: Execute agent

In [3]:
import os
import sys
import time
import json
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from agents import create_agent, invoke_agent_helper

/Users/madhurpt/Desktop/hedge-fund-analyst-multi-agent-collaboration-2
Boto3 version: 1.35.83


[2024-12-17 19:37:04,896] p85944 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:37:04,989] p85944 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-prash-none"}}}}
[2024-12-17 19:37:06,680] p85944 {utils.py:160} INFO - NumExpr defaulting to 8 threads.
[2024-12-17 19:37:07,555] p85944 {_client.py:1026} INFO - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[2024-12-17 19:37:08,178] p85944 {config.py:59} INFO - PyTorch version 2.3.0 available.
[2024-12-17 19:37:08,380] p85944 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:37:08,468] p85944 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-pr

Logged in as Weights & Biases user: madhur-prash.
View Weave data at https://wandb.ai/madhur-prash-none/multi-agent-collaboration/weave


In [4]:
# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [5]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [6]:
agent_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response.
"""

agent_description = "Agent for comprehensive financial statement analysis including income statements, balance sheets, and cash flows"

In [7]:
# This is the function definition for the lambda function that will be invoked
# as a part of the action group
functions = [{
    'name': 'get_income_statements',
    'description': 'Get income statements for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_balance_sheets',
    'description': 'Get balance sheets for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_cash_flow_statements',
    'description': 'Get cash flow statements for a company',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period of statements (ttm, quarterly, or annual)",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests` to make requests to the financial data API key.

In [8]:
# Create and publish the layer
# In this case we want to add a layer to the lambda containing files to import the requests library
layer_zip = create_lambda_layer(['requests'])
layer_arn = publish_layer('fundamental-agent-lambda-layer')

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.24 which is incompatible.
langchain-community 0.2.17 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.2.3 which is incompatible.
langchain-community 0.2.17 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
cohere 5.3.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.20.1 which is incompatible.
llam

### Create agent
---

Next we will create the agent with the provided information. In this, we will use the prompt configuration, the lambda function and lambda layers, update the lambda function to contain the `FINANCIAL_DATASETS_API_KEY` that the agent can use to fetch data to user inputs, and then finally create the agent.

In [9]:
action_group_config = {
    'name': 'FundamentalAnalysisActionGroup',
    'description': 'Action group to analyze company financial statements using income statement data',
    'functions': functions,
    'lambda_function_name': f'{SUB_AGENT_FUNDAMENTAL_ANALYST}-lambda',
    'lambda_file_path': FUNDAMENTAL_LAMBDA_FUNCTION_NAME,
    # Adding the lambda layer that contains the requests installation
    'lambda_layers': [layer_arn]
}

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = action_group_config['lambda_function_name']
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY')
}

In [10]:
# Create the fundamental analyst agent
fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, fundamental_analyst_agent_alias_arn = create_agent(
    SUB_AGENT_FUNDAMENTAL_ANALYST,
    agent_instruction,
    agent_foundation_model=BEDROCK_MODEL_CLAUDE_HAIKU,
    agent_description=agent_description,
    action_group_config=action_group_config
)

creating agent
Policy fundamental-analyst-agent-ba already exists
Checking if AmazonBedrockExecutionRoleForAgents_fundamental-analyst-agent role also exists
Detaching and deleting fundamental-analyst-agent-ba
deleting AmazonBedrockExecutionRoleForAgents_fundamental-analyst-agent
Recreating fundamental-analyst-agent-ba
creating and attaching action group
Role already exists -- deleting and creating it again
Detaching AWSLambdaBasicExecutionRole
Detaching AmazonDynamoDBFullAccess
deleting fundamental-analyst-agent-lambda-role
recreating fundamental-analyst-agent-lambda-role
attaching basic lambda permissions to fundamental-analyst-agent-lambda-role
attaching dynamodb permissions to fundamental-analyst-agent-lambda-role
fundamental-analyst-agent-lambda already exists, deleting it and recreating
Agent id WH3Y06UPJI current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id WH3Y06UPJI current status: PREPARED
Waiting for agent status to change. Curren

In [11]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=action_group_config['lambda_layers']
)

### Getting details from the agent
--- 
Let's take a look at the details from the created agent.

In [12]:
fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, fundamental_analyst_agent_alias_arn

('WH3Y06UPJI',
 '4JZJVMDGU6',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6')

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our `invoke_agent_helper` function. Here, we will invoke the `fundamental analyst` agent to provide balance sheet information, income statements and cash flow information based on the question provided by the user.

Since the `invoke_agent_helper` is wrapped with a `weave` decorator, it will log the input/outputs or any errors to your weave account dashboard. To create a weave API key, refer to the following link: https://wandb.ai/site/weave/

In [13]:
# sleep for 30 seconds before invoking the agent
time.sleep(30)

In [14]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Summarize income statements for AAPL in 2023"""
response = invoke_agent_helper(
    query, session_id, fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:39:22,762] p85944 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:39:22,763] p85944 {agents.py:761} INFO - {
  "RequestId": "9a06d10e-fe32-483a-8e1b-aa320585e782",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:39:22 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "9a06d10e-fe32-483a-8e1b-aa320585e782",
    "x-amz-bedrock-agent-session-id": "8670aa8c-bcd8-11ef-b7cc-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:39:22,976] p85944 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:39:22,976] p85944 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6"
  }
]
[2024-12-17 19:39:22,977] p85944 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d734-8107-7c90-bf52-c63ed602beb0
The key highlights from the income statements for Apple (AAPL) in 2023 are:

- Revenue: $383.285 billion, up from $365.817 billion in 2021
- Gross Profit: $169.148 billion, up from $152.836 billion in 2021
- Operating Income: $114.301 billion, up from $108.949 billion in 2021 
- Net Income: $96.995 billion, up from $94.680 billion in 2021
- Earnings Per Share (diluted): $6.13, up from $5.61 in 2021
- Dividends Per Share: $0.94, up from $0.85 in 2021

The income statement data shows continued growth in revenue, profitability, and earnings per share for Apple in 2023 compared to the prior year.
CPU times: user 72.3 ms, sys: 22.3 ms, total: 94.6 ms
Wall time: 9.66 s


As seen from above, the agent rationalizes and uses the `get_income_statements` from the `FundamentalAnalysisActionGroup` to get the summary of the income statements for `AAPL`. View the API usage of the financial dataset website below that the agent calls to get the relevant data to answer the user question.

![api-usage](images/api-usage.png)

In [15]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is AAPL's balance sheet information for the first quarter of May, 2021?"""
response = invoke_agent_helper(
    query, session_id, fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:39:32,403] p85944 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:39:32,404] p85944 {agents.py:761} INFO - {
  "RequestId": "1568b0a4-ca41-40e9-955e-24b9998f5bff",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:39:32 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "1568b0a4-ca41-40e9-955e-24b9998f5bff",
    "x-amz-bedrock-agent-session-id": "8c33c436-bcd8-11ef-b7cc-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:39:32,587] p85944 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:39:32,588] p85944 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6"
  }
]
[2024-12-17 19:39:32,588] p85944 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d734-a6c6-7982-b469-b6f7ae75af9b
I do not have the balance sheet information for Apple Inc. (AAPL) for the first quarter of 2021.
CPU times: user 76.9 ms, sys: 18.9 ms, total: 95.8 ms
Wall time: 5.67 s


In [16]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Describe AAPL's cash flow for 2024?"""
response = invoke_agent_helper(
    query, session_id, fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:39:38,077] p85944 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:39:38,078] p85944 {agents.py:761} INFO - {
  "RequestId": "c0c9e25d-4f0a-46af-865a-d5d021754cbe",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:39:38 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "c0c9e25d-4f0a-46af-865a-d5d021754cbe",
    "x-amz-bedrock-agent-session-id": "8f96040e-bcd8-11ef-b7cc-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:39:38,246] p85944 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:39:38,247] p85944 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6"
  }
]
[2024-12-17 19:39:38,248] p85944 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d734-bcf2-7c93-9989-a3bd210824d5
Based on the cash flow statement data for Apple (AAPL) in 2024:

- Net cash flow from operations was $118,254 million
- Capital expenditures were $9,447 million
- Net cash flow from investing activities was $2,935 million, including $13,690 million from investment acquisitions and disposals
- Net cash flow from financing activities was -$121,983 million, including $94,949 million from issuance or purchase of equity shares and $15,234 million in dividends and other cash distributions
- The overall change in cash and equivalents was -$794 million
- The ending cash balance was $29,943 million
- The free cash flow (net cash flow from operations minus capital expenditures) was $108,807 million

So in summary, Apple generated strong operating cash flow in 2024 but used a significant amount of cash for financing activities like share repurchases and dividends, resulting in a small net de

In [17]:
%%time
import uuid
session_id:str = str(uuid.uuid1())

# testing the agent with an unrelated question
query = """What is a llama?"""
response = invoke_agent_helper(
    query, session_id, fundamental_analyst_agent_id, fundamental_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:39:42,849] p85944 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:39:42,849] p85944 {agents.py:761} INFO - {
  "RequestId": "a55b6782-f985-4c9c-8776-94847dd07ce0",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:39:42 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "a55b6782-f985-4c9c-8776-94847dd07ce0",
    "x-amz-bedrock-agent-session-id": "926cf5ac-bcd8-11ef-b7cc-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:39:43,017] p85944 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:39:43,018] p85944 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6"
  }
]
[2024-12-17 19:39:43,019] p85944 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d734-cf8f-79b2-9192-79ca4eb3683f
I apologize, but I do not have any information about what a llama is. As an AI assistant, I do not have general knowledge about animals or other topics outside of my training. I can only provide information related to the financial analysis tasks I was designed for. If you have a question about analyzing a company's financial statements, I would be happy to assist with that. Otherwise, I do not have the capability to answer your question about llamas.
CPU times: user 48.2 ms, sys: 7.16 ms, total: 55.3 ms
Wall time: 1.74 s


### Saving the information
---
Let's now save some information so that we can use this agent as a sub-agent of our financial hedge fund assistant

In [18]:
%store fundamental_analyst_agent_id
%store fundamental_analyst_agent_alias_id
%store fundamental_analyst_agent_alias_arn

Stored 'fundamental_analyst_agent_id' (str)
Stored 'fundamental_analyst_agent_alias_id' (str)
Stored 'fundamental_analyst_agent_alias_arn' (str)
